In [1]:
import yt
import numpy as np
import os
import pandas as pd

In [2]:
curr_dir = os.getcwd()
all_files = os.listdir(curr_dir+"/mu_-0.2/")
for filename in all_files:
    if filename.startswith("inputs"):
        input_file = filename
input_file = curr_dir+"/mu_-0.2/"+input_file
fo = open(input_file, "r")

In [3]:
def extract_parameters(fo):
    parameters = dict()
    for line in fo.readlines():
        if line.startswith("nsteps"):
            temp = line.split()
            parameters["nL"] = int(temp[-1])
        elif line.startswith("n_cell"):
            temp = line.split()
            parameters["Nx"] = int(temp[2])
            parameters["Nt"] = int(temp[-1])
        elif line.startswith("nrrb.l"):
            temp = line.split()
            parameters["lambda"] = float(temp[-1])
        elif line.startswith("nrrb.mu"):
            temp = line.split()
            parameters["mu"] = float(temp[-1])
        elif line.startswith("nrrb.m"):
            temp = line.split()
            parameters["m"] = float(temp[-1])
        elif line.startswith("nrrb.w"):
            temp = line.split()
            parameters["wz"] = float(temp[-1])
        elif line.startswith("nrrb.w_t"):
            temp = line.split()
            parameters["wtr"] = float(temp[-1])
        elif line.startswith("nrrb.dtau"):
            temp = line.split()
            parameters["dt"] = float(temp[-1])
        elif line.startswith("nrrb.eps"):
            temp = line.split()
            parameters["eps"] = float(temp[-1])
    parameters["beta"] = parameters["dt"]*parameters["Nt"]
    return parameters

parameters = extract_parameters(fo)
print(parameters)

{'Nt': 80, 'Nx': 21, 'wz': 0.0, 'beta': 4.0, 'dt': 0.05, 'mu': -0.2, 'nL': 100000, 'eps': 0.001, 'lambda': 0.0, 'm': 1.0}


In [4]:
nL = parameters['nL']
all_steps = np.arange(1,nL+1,1)
all_plotfiles = []
for pltfile in all_files:
    if pltfile.startswith("plt"):
        all_plotfiles.append(int(pltfile[3:]))
#print(all_plotfiles)

In [5]:
def find_missing_files(all_steps, all_plotfiles):
    missing = []
    for step in all_steps:
        if step not in all_plotfiles:
            missing.append(step)
    return missing

In [6]:
#force nL to be smaller
#nL = len(all_plotfiles)
nL = 20
missing = find_missing_files(np.arange(1,nL+1), all_plotfiles)
print(len(missing))

0


In [7]:
def get_fields(ds,i,j,t):
    cg = ds.covering_grid(left_edge=ds.domain_left_edge, dims=ds.domain_dimensions, level=0)
    coords = cg.fcoords

    xs = coords[:,0]
    ys = coords[:,1]
    ts = coords[:,2]

    dx = np.max(xs[1:] - xs[:-1])
    dy = np.max(ys[1:] - ys[:-1])
    dt = np.max(ts[1:] - ts[:-1])

    phi_1_Re = cg["phi_1_Re"][:,:,:].d
    phi_1_Im = cg["phi_1_Im"][:,:,:].d
    phi_2_Re = cg["phi_2_Re"][:,:,:].d
    phi_2_Im = cg["phi_2_Im"][:,:,:].d
    
    return complex(phi_1_Re[i,j,t], phi_1_Im[i,j,t]), complex(phi_2_Re[i,j,t], phi_2_Im[i,j,t])

In [8]:
work_dir = curr_dir+"/mu_-0.2/"
filename = "plt"+str(all_plotfiles[0]).zfill(5) 
ds = yt.load(work_dir+filename)

yt : [INFO     ] 2020-01-14 10:52:33,808 Parameters: current_time              = 1.82399999999991
yt : [INFO     ] 2020-01-14 10:52:33,810 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 10:52:33,812 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 10:52:33,815 Parameters: domain_right_edge         = [21. 21. 80.]


In [9]:
get_fields(ds,0,0,0)

((0.837475527894637-0.15995733088335284j),
 (2.2544157081884784-0.4527482314410909j))

Average density: $\langle \hat{n} \rangle = \frac{1}{N_{x}^{d}}\sum_{x}n_{x}$

where $n_{x}$ is the local density: $n_{x}= \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}-i \phi_{2,x})(\phi_{1,x-\hat{\tau}}+i \phi_{2,x-\hat{\tau}})$

Using our four complexified fields:
\begin{eqnarray}
n_{x} &=& \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}^{R} + i \phi_{1,x}^{I}-i \phi_{2,x}^{R} +\phi_{2,x}^{I})(\phi_{1,x-\hat{\tau}}^{R} + i \phi_{1,x-\hat{\tau}}^{I} +i \phi_{2,x-\hat{\tau}}^{R} - \phi_{2,x-\hat{\tau}}^{I})\\
&=& \frac{1}{2 N_{\tau}}e^{\beta \mu/N_{\tau}}(\phi_{1,x}^{R} + i \phi_{1,x}^{I}-i \phi_{2,x}^{R} +\phi_{2,x}^{I})(\phi_{1,x-\hat{\tau}}^{R} + i \phi_{1,x-\hat{\tau}}^{I} +i \phi_{2,x-\hat{\tau}}^{R} - \phi_{2,x-\hat{\tau}}^{I})
\end{eqnarray}

Question for self: do you need to average over all t?

Figure out how to deal with boundary conditions

In [14]:
def local_density(ds, i, j, Nt, beta, mu):
    n = complex(0.,0.)
    #for t in np.arange(1,Nt):
    #    phi1,phi2 = get_fields(ds,i,j,t)
    #    phi1t,phi2t = get_fields(ds,i,j,t-1)
    #    n += complex(phi1, -phi2)*complex(phi1t, -phi2t)
    #phi1,phi2 = get_fields(ds,i,j,0)
    #phi1t,phi2t = get_fields(ds,i,j,Nt-1)
    t = int(Nt/2)
    phi1,phi2 = get_fields(ds,i,j,t)
    phi1t,phi2t = get_fields(ds,i,j,t-1)
    n += complex(phi1, -phi2)*complex(phi1t, -phi2t)
    return 0.5*np.exp(-beta*mu/float(Nt))*n/float(Nt)

def total_density(ds, Nx, Nt, beta, mu):
    ntot = complex(0.,0.)
    volume = Nx**2
    for i in np.arange(0,Nx):
        for j in np.arange(0,Nx):
            ntot += local_density(ds, i, j, Nt, beta, mu)
    return ntot/volume

In [ ]:
nL = 20
Nx = parameters['Nx']
Nt = parameters['Nt']
beta = parameters['beta']
mu = parameters['mu']
df = pd.DataFrame(columns=['step','N'])
df.head()
work_dir = curr_dir+"/mu_-0.2/"
for step in np.arange(1,nL+1):
    filename = "plt"+str(step).zfill(5) 
    ds = yt.load(work_dir+filename)
    N = total_density(ds, Nx, Nt, beta, mu)
    df.append({'step': step, 'density': N}, ignore_index=True)
    print(filename)
    print(N)
df.head()

yt : [INFO     ] 2020-01-14 12:55:55,598 Parameters: current_time              = 0.001
yt : [INFO     ] 2020-01-14 12:55:55,599 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 12:55:55,602 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 12:55:55,603 Parameters: domain_right_edge         = [21. 21. 80.]
yt : [INFO     ] 2020-01-14 12:56:37,880 Parameters: current_time              = 0.002
yt : [INFO     ] 2020-01-14 12:56:37,880 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 12:56:37,881 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 12:56:37,884 Parameters: domain_right_edge         = [21. 21. 80.]


plt00001
(-0.00016403912781228996-0.00013143289892008812j)


yt : [INFO     ] 2020-01-14 12:57:18,953 Parameters: current_time              = 0.003
yt : [INFO     ] 2020-01-14 12:57:18,954 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 12:57:18,955 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 12:57:18,956 Parameters: domain_right_edge         = [21. 21. 80.]


plt00002
(-0.00014232934779516833-0.00011799791785626334j)


yt : [INFO     ] 2020-01-14 12:58:00,102 Parameters: current_time              = 0.004
yt : [INFO     ] 2020-01-14 12:58:00,103 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 12:58:00,104 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 12:58:00,105 Parameters: domain_right_edge         = [21. 21. 80.]


...

plt00015
(-0.00016556508144478953-0.00020058576858744808j)


yt : [INFO     ] 2020-01-14 13:08:55,818 Parameters: current_time              = 0.017000000000000008
yt : [INFO     ] 2020-01-14 13:08:55,819 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 13:08:55,820 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 13:08:55,823 Parameters: domain_right_edge         = [21. 21. 80.]


plt00016
(-0.0001452359692240056-0.0002204702847572037j)


yt : [INFO     ] 2020-01-14 13:09:59,683 Parameters: current_time              = 0.01800000000000001
yt : [INFO     ] 2020-01-14 13:09:59,684 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-01-14 13:09:59,685 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-01-14 13:09:59,686 Parameters: domain_right_edge         = [21. 21. 80.]


plt00017
(-0.00014739433173498092-0.00021760067865479163j)
